# Carregamento e Inspeção

In [1]:
import pandas as pd
import numpy as np

# 1. Carregando o arquivo
# Encoding 'latin1' é um bom backup, caso o padrão UTF-8 falhe
try:
  df = pd.read_csv("/content/Online_Retail.csv", encoding='latin1')
except FileNotFoundError:
  print("ERRO: O arquivo '/content/Online_Retail.csv' não foi encontrado.")

print("carregamento e primeiras linhas:")
display(df.head())
print("\nVerificando tipos de dados e valores ausentes (NaN):")
df.info()

carregamento e primeiras linhas:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom



Verificando tipos de dados e valores ausentes (NaN):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


# Limpeza Essencial e Métrica Total

In [3]:
# 1. Limpeza de Valores Ausentes (NaN)
# CustomerID é essencial para a segmentação RFM
df.dropna(subset=['CustomerID', 'Description'], inplace=True)

In [4]:
# 2. Remoção de Transações Inválidas
# Pedidos com Quantity < 0 são cancelamentos. Pedidos com UnitPrice <= 0 são itens promocionais ou erros
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]

In [5]:
# 3. Conversão da Data
# O formato de data/hora (datetime) é crucial para o cálculo da Recência (R)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

/tmp/ipython-input-739084439.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


In [7]:
# 4. Criação da Métrica Total de Transação (TotalPrice)
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Convertendo CustomerID para string (o ID do cliente é uma chave, não um número para cálculo)
df['CustomerID'] = df['CustomerID'].astype(int).astype(str)

print(f"\nDataFrame limpo pronto para análise com {len(df):,} transações")
display(df.head())


DataFrame limpo pronto para análise com 397,884 transações


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


# Calculando o RFM
R (Recency/Recência): Há quantos dias o cliente comprou pela última vez. (Quanto menor, melhor)

F (Frequency/Frequência): Quantas vezes o cliente comprou. (Quanto maior, melhor)

M (Monetary/Valor Monetário): Quanto o cliente gastou no total. (Quanto maior, melhor)

In [8]:
import datetime as dt

# Encontrando a última data de compra no dataset
ultima_data = df['InvoiceDate'].max()

# Definindo a data de referência (SnapshotDate) como 1 dia após a última compra
SnapshotDate = ultima_data + dt.timedelta(days=1)

# Agrupando por cliente (CustomerID) para calcular o RFM
df_rfm = df.groupby('CustomerID').agg(
    # Recência (R): A diferença em dias entre a data de referência e a última compra
    Recency=('InvoiceDate', lambda x: (SnapshotDate - x.max()).days),

    # Frequência (F): Contagem de faturas únicas
    Frequency=('InvoiceNo', 'nunique'),

    # Valor Monetário (M): Soma do valor total gasto
    Monetary=('TotalPrice', 'sum')
).reset_index()

print("--- Tabela de Análise RFM Pronta ---")
display(df_rfm.head())

print(f"\nClientes únicos analisados: {len(df_rfm):,}")

--- Tabela de Análise RFM Pronta ---


,CustomerID,Recency,Frequency,Monetary
0,12346,326,1,77183.60
1,12347,2,7,4310.00
2,12348,75,4,1797.24
3,12349,19,1,1757.55
4,12350,310,1,334.40



Clientes únicos analisados: 4,338


# Atribuindo Scores de 1 a 5


In [11]:
# 1. Pontuação de Frequência (F) e Valor Monetário (M)
# Pontuação 5 é dada aos valores mais altos (melhores)
df_rfm['F_Score'] = pd.qcut(df_rfm['Frequency'], 5, labels=False, duplicates='drop') + 1
df_rfm['M_Score'] = pd.qcut(df_rfm['Monetary'], 5, labels=False, duplicates='drop') + 1

# 2. Pontuação de Recência (R)
# A Recência deve ser INVERTIDA: Pontuação 5 é dada aos valores mais baixos (Recência mais recente)
df_rfm['R_Score']= pd.qcut(df_rfm['Recency'], 5, labels=False, duplicates='drop') + 1
# Invertendo o score da Recência (5 deve ser o menor número de dias)
df_rfm['R_Score'] = df_rfm['R_Score'].apply(lambda x: 6 - x)

# 3. Criando o Score Combinado (RFM_Score)
# O score total é a concatenação das pontuações R, F e M (ex: 555, 453)
df_rfm['RFM_Score'] = df_rfm['R_Score'].astype(str) + df_rfm['F_Score'].astype(str) + df_rfm['M_Score'].astype(str)

print("\n--- Tabela RFM com scores pronta ---")
display(df_rfm.head())


--- Tabela RFM com scores pronta ---


,CustomerID,Recency,Frequency,Monetary,F_Score,M_Score,R_Score,RFM_Score
0,12346,326,1,77183.60,1,5,1,115
1,12347,2,7,4310.00,4,5,5,545
2,12348,75,4,1797.24,3,4,2,234
3,12349,19,1,1757.55,1,4,4,414
4,12350,310,1,334.40,1,2,1,112


## Criando Segmentos de Negócio

In [12]:
# Função para atribuir o rótulo do segmento baseado nos scores R e F
def segmentar_cliente(row):
  # Clientes Campeões (Champions)
  if row['R_Score'] >= 4 and row['F_Score'] >= 4:
    return '01. Clientes Campeões'

  # Clientes Leais (Loyal Customers)
  elif row['F_Score'] >= 4 and row['R_Score'] >= 2:
    return '02. Clientes Leais'

 # Novos Clientes (New Customers)
  elif row['R_Score'] == 5 and row['F_Score'] <= 2:
    return '03. Novos Clientes'

 # Clientes Promissores (Potential Loyalists)
  elif row['R_Score'] >= 3 and row['F_Score'] >= 1:
    return '04. Clientes Promissores'

 # Clientes em Risco (At Risk)
  elif row['R_Score'] <= 2 and row['F_Score'] >=3:
    return '05. Clientes em Risco'

# Não Dormem (Can’t Lose Them)
  elif row['R_Score'] <= 1 and row['F_Score'] >= 4:
    return '06. Não Dormem'

# Hibernando (Hibernating)
  elif row['R_Score'] <=2 and row['F_Score'] <= 2:
    return '07. Hibernando'

# Perdidos (Lost) - os piores em R e F
  else:
    return '08. Clientes Perdidos'

# Aplicando a função para criar a coluna Segmento
df_rfm['Segmento'] = df_rfm.apply(segmentar_cliente, axis=1)

print("\n--- Distribuição dos Segmentos (o insight Principal) ---")
display(df_rfm['Segmento'].value_counts().reset_index())


--- Distribuição dos Segmentos (o insight Principal) ---


,Segmento,count
0,04. Clientes Promissores,1701
1,07. Hibernando,1505
2,01. Clientes Campeões,578
3,03. Novos Clientes,264
4,05. Clientes em Risco,175
5,02. Clientes Leais,115


In [ ]:
# O maior grupo são os Clientes Promissores (1.711) e o segundo maior são os Hibernando (1.505).
# Isso indica que a empresa tem uma grande base de clientes que já interagiu, mas que ou está no começo do ciclo de vida (Promissores) ou está escorregando para o churn (Hibernando).

# A estratégia de marketing da empresa deve ser focada em converter clientes Promissores em Campeões e reativar clientes Hibernando para evitar a perda da base.

# Agregação para o Heatmap

In [13]:
# Agrupa pela pontuação de Recência e Frequência, e calcula o valor médio de M (Monetary)
df_rfm_heatmap = df_rfm.groupby(['R_Score', 'F_Score']).agg(
    Monetary_Mean=('Monetary', 'mean'), # O valor médio gasto por esse segmento R-F
    Clientes =('CustomerID', 'count')   # Quantidade de clientes nesse segmento R-F
).reset_index()

print("\n--- Tabela Pronta para Heatmap R-F (Média Monetária) ---")
display(df_rfm_heatmap.head())


--- Tabela Pronta para Heatmap R-F (Média Monetária) ---


,R_Score,F_Score,Monetary_Mean,Clientes
0,1,1,485.422598,778
1,1,2,2507.889608,51
2,1,3,997.486552,29
3,1,4,3505.867143,7
4,2,1,549.120331,547


# Exportação e Preparação para o Dashboard

In [14]:
# Exporta a tabela principal RFM para um CSV
df_rfm.to_csv('rfm_segmentacao_final.csv', index=False)